In [2]:
import pandas as pd

trip_data_path = '/content/metro-trips-2024-q3.csv'
cleaned_station_data_path = '/content/metro-bike-share-stations-2024.csv'


bike_data = pd.read_csv(trip_data_path)
stations_data = pd.read_csv(cleaned_station_data_path)

# Drop rows with missing latitude/longitude
rows_before = bike_data.shape[0]
bike_data = bike_data.dropna(subset=['start_lat', 'start_lon', 'end_lat', 'end_lon'])
rows_after = bike_data.shape[0]
print(f"Rows before: {rows_before}")
print(f"Rows after: {rows_after}")
print(f"Amount of rows dropped: {rows_before - rows_after}")

# Correcting data types to datetime
bike_data['start_time'] = pd.to_datetime(bike_data['start_time'], errors='coerce')
bike_data['end_time'] = pd.to_datetime(bike_data['end_time'], errors='coerce')

# Standardize text columns
bike_data['passholder_type'] = bike_data['passholder_type'].str.strip().str.lower()
bike_data['trip_route_category'] = bike_data['trip_route_category'].str.strip().str.lower()
bike_data['bike_type'] = bike_data['bike_type'].str.strip().str.lower()

# Merge data to add start kiosk and region
bike_data = bike_data.merge(
    stations_data[['Kiosk ID', 'Kiosk Name', 'Region ']].rename(columns={
        'Kiosk ID': 'start_station',
        'Kiosk Name': 'start_kiosk_name',
        'Region ': 'start_region'
    }),
    on='start_station',
    how='left'
)

# Merge data to add end kiosk and region
bike_data = bike_data.merge(
    stations_data[['Kiosk ID', 'Kiosk Name', 'Region ']].rename(columns={
        'Kiosk ID': 'end_station',
        'Kiosk Name': 'end_kiosk_name',
        'Region ': 'end_region'
    }),
    on='end_station',
    how='left'
)

# Identify invalid start and end stations
invalid_start_stations = bike_data[bike_data['start_kiosk_name'].isnull()]
invalid_end_stations = bike_data[bike_data['end_kiosk_name'].isnull()]

print(f"Invalid start stations count: {len(invalid_start_stations)}")
print(f"Invalid end stations count: {len(invalid_end_stations)}")

# Drop rows with invalid start or end stations
rows_before_validation = bike_data.shape[0]
bike_data = bike_data[bike_data['start_kiosk_name'].notnull() & bike_data['end_kiosk_name'].notnull()]
rows_after_validation = bike_data.shape[0]
print(f"Rows before validation: {rows_before_validation}")
print(f"Rows after validation: {rows_after_validation}")
print(f"Invalid trips dropped: {rows_before_validation - rows_after_validation}")

# Droping unneeded columns
columns_to_drop = ['bike_id', 'plan_duration']
bike_data.drop(columns=[col for col in columns_to_drop if col in bike_data.columns], inplace=True)


clean_file_path = '/content/cleaned-metro-trips-2024-q3.csv'
bike_data.to_csv(clean_file_path, index=False)

print(f"Cleaned trip data saved to: {clean_file_path}")

print("\nTrip Data Columns and Data Types:")
print(bike_data.dtypes)


FileNotFoundError: [Errno 2] No such file or directory: '/content/metro-trips-2024-q3.csv'